<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Loading-dependencies" data-toc-modified-id="Loading-dependencies-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Loading dependencies</a></span></li><li><span><a href="#Reading-and-Understanding-the-data" data-toc-modified-id="Reading-and-Understanding-the-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Reading and Understanding the data</a></span></li><li><span><a href="#Data-Preparation" data-toc-modified-id="Data-Preparation-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Data Preparation</a></span><ul class="toc-item"><li><span><a href="#Handling-Missing-data" data-toc-modified-id="Handling-Missing-data-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Handling Missing data</a></span></li><li><span><a href="#Examine-each-column" data-toc-modified-id="Examine-each-column-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Examine each column</a></span></li><li><span><a href="#Column-level-standardisation-(for-e.g.,-date,-etc.)" data-toc-modified-id="Column-level-standardisation-(for-e.g.,-date,-etc.)-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Column level standardisation (for e.g., date, etc.)</a></span></li><li><span><a href="#Convert-numeric-values-into-categorical-string-values" data-toc-modified-id="Convert-numeric-values-into-categorical-string-values-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Convert numeric values into categorical string values</a></span></li></ul></li><li><span><a href="#Exploratory-Data-Analysis" data-toc-modified-id="Exploratory-Data-Analysis-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Exploratory Data Analysis</a></span><ul class="toc-item"><li><span><a href="#Visualising-Numeric-Variables" data-toc-modified-id="Visualising-Numeric-Variables-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Visualising Numeric Variables</a></span></li></ul></li></ul></div>

## Loading dependencies

In [29]:
# Supress warnings
import warnings
warnings.filterwarnings('ignore')

In [30]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import xticks
from haversine import haversine
from datetime import datetime
from collections import Counter
from imblearn.over_sampling import SMOTE
from numpy import where
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score, precision_recall_curve, confusion_matrix, plot_roc_curve, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import classification_report
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

# Importing required packages for visualization
from IPython.display import Image  
from six import StringIO  
from sklearn.tree import export_graphviz
import pydotplus, graphviz

In [31]:
# Display all columns
pd.set_option('display.max_columns',200)

## Reading and Understanding the data

In [32]:
data = pd.read_csv("train.csv")

print(data.shape)

(567545, 11)


In [33]:
# Read data
data.head()

,order_id,service_type,driver_status,date,hour,seconds,latitude,longitude,altitude_in_meters,accuracy_in_meters,label
0,RB193,GO_RIDE,UNAVAILABLE,2018-02-05,6,1548890667,-6.922910,107.631301,NaN,23.027,0
1,RB193,GO_RIDE,AVAILABLE,2018-02-05,6,1548890680,-6.923039,107.631250,712.0,9.577,0
2,RB193,GO_RIDE,AVAILABLE,2018-02-05,6,1548890690,-6.923039,107.631250,712.0,9.577,0
3,RB193,GO_RIDE,AVAILABLE,2018-02-05,6,1548890700,-6.923048,107.631230,713.0,8.139,0
4,RB193,GO_RIDE,AVAILABLE,2018-02-05,6,1548890710,-6.922968,107.631253,713.0,7.029,0


In [34]:
# Types of all columns
data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 567545 entries, 0 to 567544
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   order_id            567545 non-null  object 
 1   service_type        567545 non-null  object 
 2   driver_status       567545 non-null  object 
 3   date                567545 non-null  object 
 4   hour                567545 non-null  int64  
 5   seconds             567545 non-null  int64  
 6   latitude            567545 non-null  float64
 7   longitude           567545 non-null  float64
 8   altitude_in_meters  413142 non-null  float64
 9   accuracy_in_meters  567545 non-null  float64
 10  label               567545 non-null  int64  
dtypes: float64(4), int64(3), object(4)
memory usage: 47.6+ MB


In [35]:
# Check the summary for the numeric columns 
data.describe()

,hour,seconds,latitude,longitude,altitude_in_meters,accuracy_in_meters,label
count,567545.000000,5.675450e+05,567545.000000,567545.000000,413142.000000,567545.000000,567545.000000
mean,14.435453,1.550739e+09,-6.902215,107.594670,744.885056,31.031366,0.719034
std,4.942749,1.123543e+06,0.141004,0.110761,105.357303,199.671686,0.449471
min,0.000000,1.548891e+09,-66.655409,63.592748,-1075.000000,0.010000,0.000000
25%,11.000000,1.549779e+09,-6.922561,107.551299,708.300000,0.500000,0.000000
50%,15.000000,1.550658e+09,-6.893074,107.603726,748.400000,4.133000,1.000000
75%,18.000000,1.551677e+09,-6.878771,107.627953,790.481659,10.380000,1.000000
max,23.000000,1.552855e+09,-6.804238,107.776651,4674.877000,9628.102000,1.000000


## Data Preparation

### Handling Missing data

In [36]:
# List of columns which have null values
missing_data_percent = 100*data.isnull().sum()/len(data)
missing_data_percent[missing_data_percent>0].sort_values(ascending = False)

altitude_in_meters    27.20542
dtype: float64

In [37]:
# Count the number of null values in each column
data.isnull().sum()

order_id                   0
service_type               0
driver_status              0
date                       0
hour                       0
seconds                    0
latitude                   0
longitude                  0
altitude_in_meters    154403
accuracy_in_meters         0
label                      0
dtype: int64

### Examine each column

In [38]:
# View values of all columns
for c in data:
    print(data[c].value_counts())
    print("\n\n==============\n\n")

RB622    685
RB82     667
F985     660
F1685    656
F1160    600
        ... 
RB952      3
RB637      2
F803       2
F1462      1
F601       1
Name: order_id, Length: 3500, dtype: int64




GO_FOOD    335863
GO_RIDE    231682
Name: service_type, dtype: int64




OTW_DROPOFF    251330
AVAILABLE      175896
OTW_PICKUP     135668
UNAVAILABLE      4651
Name: driver_status, dtype: int64




2018-02-06    17174
2018-02-22    15960
2018-02-16    15882
2018-02-21    15744
2018-02-11    15700
2018-02-14    15663
2018-02-19    15551
2018-02-25    15383
2018-02-13    14967
2018-02-08    13891
2018-02-20    13718
2018-03-04    13671
2018-03-01    13591
2018-03-11    13492
2018-02-05    13419
2018-02-12    13183
2018-02-27    13025
2018-03-08    12971
2018-03-07    12941
2018-02-23    12852
2018-02-07    12739
2018-02-15    12208
2018-03-05    11846
2018-02-24    11842
2018-03-10    11835
2018-03-09    11641
2018-02-26    11625
2018-03-21    11168
2018-02-18    11122
2018-03-03    11037
2018-02-17 

=> Consider replacing Nan of altitude_in_meters with 0

=> Check 0 value in all columns:

In [39]:
# Count zeros. NaN is NOT considered zero
data.isin([0]).astype(int).sum(axis=0)

order_id                   0
service_type               0
driver_status              0
date                       0
hour                    7233
seconds                    0
latitude                   0
longitude                  0
altitude_in_meters         0
accuracy_in_meters         0
label                 159461
dtype: int64

=> 7233 orders were requested at mid-night. 

### Column level standardisation (for e.g., date, etc.)

In [40]:
# Convert Linux seconds to datetime format
data['linux_date'] = [datetime.utcfromtimestamp(s).strftime('%Y-%m-%d %H:%M:%S') for s in data.seconds.values]

# Convert datetime to Pandas format
data['linux_date'] = pd.to_datetime(data['linux_date'])

# Convert datetime in date column to Pandas format
data['date'] = pd.to_datetime(data['date'])

# Check if date column match with Linux date column
df = data['linux_date'].dt.date==data['date']
print(df.eq(True).all())

False


In [41]:
data.head()

,order_id,service_type,driver_status,date,hour,seconds,latitude,longitude,altitude_in_meters,accuracy_in_meters,label,linux_date
0,RB193,GO_RIDE,UNAVAILABLE,2018-02-05,6,1548890667,-6.922910,107.631301,NaN,23.027,0,2019-01-30 23:24:27
1,RB193,GO_RIDE,AVAILABLE,2018-02-05,6,1548890680,-6.923039,107.631250,712.0,9.577,0,2019-01-30 23:24:40
2,RB193,GO_RIDE,AVAILABLE,2018-02-05,6,1548890690,-6.923039,107.631250,712.0,9.577,0,2019-01-30 23:24:50
3,RB193,GO_RIDE,AVAILABLE,2018-02-05,6,1548890700,-6.923048,107.631230,713.0,8.139,0,2019-01-30 23:25:00
4,RB193,GO_RIDE,AVAILABLE,2018-02-05,6,1548890710,-6.922968,107.631253,713.0,7.029,0,2019-01-30 23:25:10


### Convert numeric values into categorical string values

In [43]:
data['label'] = data['label'].map({1: 'Yes', 0: 'No'})

## Exploratory Data Analysis

### Visualising Numeric Variables

In [42]:
# Show all numerical columns
data.describe().columns

Index(['hour', 'seconds', 'latitude', 'longitude', 'altitude_in_meters',
       'accuracy_in_meters', 'label'],
      dtype='object')

In [ ]:
# Correlation between numeric variables
cor = data[['hour', 'seconds', 'latitude', 'longitude', 'altitude_in_meters',
       'accuracy_in_meters', 'label']].corr()
cor